GLF Kernel - Tutorial
===

#### Installation
See the instructions in the GitHub repository ([https://github.com/kwarc/glf_kernel](https://github.com/kwarc/glf_kernel)).

#### Very Brief Intro: Jupyter Cells
In Jupyter, there are two types of cells:
* *markdown cells* (such as this one), which can be used for documentation and explanations
* *code cells*, which contain GF grammars and shell commands if you use the GF kernel

You can edit a cell by clicking on it (double clicking in the case of markdown cells).
To execute the cell, you can press <kbd>Ctrl</kbd>+<kbd>Return</kbd> or <kbd>Shift</kbd>+<kbd>Return</kbd>. If you want to execute all cells in the notebook, you can choose *Cell* -> *Run All* in the top menu.

## GLF Kernel Commands

You can use this kernel to create new MMT archives or switch to existing ones using the `archive` command.

`archive -c 'archive-name'` will create a new archive.

`archive 'archive-name'` will set your current working archive to the speciified archive.

The default archive for this kernel will be `COMMA/JUPYTER`

In [1]:
archive -c comma/jupyter

Created comma/jupyter

## Defining Grammars
Now you can define GF Grammars directly in the notebook using code cells.

These grammars will automatically be saved into the source directory of the specified archive.

For toggling line numbers in a cell, you need to switch to command mode (<kbd>Esc</kbd>) and press <kbd>L</kbd>.

In [2]:
abstract Grammar = {
    cat
        Person ; Action ; Sentence ;

    fun
        john, mary : Person ;
        run, be_happy : Action ;
        make_sentence : Person -> Action -> Sentence ;
        and : Sentence -> Sentence -> Sentence ;
}

Defined Grammar

In [3]:
concrete GrammarEng of Grammar = {
    lincat
        Person = Str ;
        Action = Str ;
        Sentence = Str ;
    lin
        john = "John" ;
        mary = "Mary" ;
        run = "runs" ;
        be_happy = "is happy" ;
        make_sentence person action = person ++ action ;
        and a b = a ++ "and" ++ b ;
}

Defined GrammarEng

If you also want to have one of your grammars as a `.gf` file, you can export it with the kernel command `export`.

In [4]:
export Grammar

Exported Grammar.gf

You can also import a grammar from a file using the GF shell command `import`.

*Note*: For importing `ExampleGer.gf` it is required that `Example.gf` exists, so you need to `export Example` first.

In [10]:
import ExampleGer.gf

Abstract changed, previous concretes discarded.


You can also use it to define new theories and views that you would like to use with your grammars.
Every time a new theory, view or grammar is specified MMT will build your current archive.

In [5]:
namespace http://mathhub.info/COMMA/JUPYTER ❚

theory FolLogic : ur:?LF =
	o : type ❙
	ι : type ❙
	and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 ❙
❚

Created theory FolLogic

In [6]:
namespace http://mathhub.info/COMMA/JUPYTER ❚

theory DomainTheory : ?FolLogic =
	john : ι ❙
	mary : ι ❙
	run : ι ⟶ o ❙
	happy : ι ⟶ o ❙
❚

Created theory DomainTheory

In [7]:
namespace http://mathhub.info/COMMA/JUPYTER ❚

view grammarSem : JUPYTER/Grammar.gf?Grammar -> ?DomainTheory =
	Person = ι ❙
	Action = ι ⟶ o ❙
	Sentence = o ❙
	john = john ❙
	mary = mary ❙
	run = run ❙
	be_happy = happy ❙
	make_sentence = [p,a] (a p) ❙
	and = [a,b] b ∧ a ❙
❚

Created view grammarSem

Now you can send requests to the GLF-Server using the `request` command.
By default `request` uses the last view you defined and a takes list of abstract syntax trees separated by vertical bars as argument.

In [8]:
request and (make_sentence john run) (make_sentence mary be_happy) | make_sentence john run

["(happy mary)∧(run john)","run john"]

Should you wish to change the current view that is used for the requests you can do it via the -v option.
This also sets the specified view as the default view for further requests.

In [ ]:
request -v grammarSem2 and (make_sentence john run) (make_sentence mary be_happy) | make_sentence john run

## Using GF Shell Commands

You can use all the GF shell commands you are used to by simply entering them in a code cell:

In [14]:
p -lang=Eng "John loves Mary" | l -lang=Ger

Category S is not in scope


#### Viewing Graphs

You can view graphs in an external application as you are used to (e.g. `visualize_tree -view=MyImageViewer`).
However, it is also possible, to view graphs directly in the notebook.
For this, the command needs to be preceded by the kernel instruction `view`.

In [6]:
view parse -lang=Eng "John loves Mary" | visualize_parse

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x13\x00\x00\x00\xfb\x08\x06\x00\x00\x00I\xc9p\xe…

If multiple graphs are generated, you can switch between them in a drop-down menu:

In [7]:
view parse -lang=Eng "John loves Mary and Mary loves John and John loves John" | vt

Dropdown(description='Tree of:', options=('and (love john mary) (and (love mary john) (love john john))', 'and…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x028\x00\x00\x01[\x08\x06\x00\x00\x00\xd8\xfd\xf4\x9…

## Further Kernel Commands

Apart from `export` and `view`, there are two more kernel commands: `clean` and `help`.

`clean` removes `.gfo`, `.dot` and `.png` files.

`help` displays information on the kernel commands (not the GF shell commands).
If you want information on the GF shell commands instead, you should use
the shortcut `h` instead:

In [8]:
h reload

r, reload
repeat the latest import command
